In [6]:
# Function that tests whether two transition matrices are equal
# Null hypothesis: empirical transition matrices arise from the same population
# Input: two matrices A, B with transition counts
# Output: array with two entries: 
#         [0]: p-value for null hypothesis (via likelihood ratio test)
#         [1]: Vovk-Sellke lower bound on prob(null hypothesis given the data)
import warnings
import numpy as np
from scipy.stats import chi2
from sklearn.preprocessing import normalize
warnings.filterwarnings("ignore")

def compareTransitions(A,B):
    # under hyp. trans. matrices are equal
    dimH0 = A.shape[0]*(A.shape[0]-1)
    C = np.ndarray.flatten(A+B)
    C = C[C>0]
    Q = np.ndarray.flatten(normalize(A+B, norm='l1', axis=0))
    Q = Q[Q>0]  
    loglikeH0 = np.inner(C,np.log(Q))
    # under hyp. trans. matrices are different
    dimH1 = 2*dimH0
    CA = np.ndarray.flatten(A)
    CA = CA[CA>0]
    QA = np.ndarray.flatten(normalize(A, norm='l1', axis=0))
    QA = QA[QA>0]
    CB = np.ndarray.flatten(B)
    CB = CB[CB>0]
    QB = np.ndarray.flatten(normalize(B, norm='l1', axis=0))
    QB = QB[QB>0]
    loglikeH1 = np.inner(CA,np.log(QA))+np.inner(CB,np.log(QB))
    # compute test stat, p-value & Vovk-Sellke bound
    teststat = -2*(loglikeH0-loglikeH1)
    pval = 1-chi2.cdf(teststat,dimH0)
    vovk = pval
    if pval < np.exp(-1):
        bound = -np.exp(1)*pval*np.log(pval)
        if pval == 0:
            vovk = 0
        else:
            vovk = 1/(1+1/bound) 
    res = np.array([pval,vovk])
    return res

In [7]:
import numpy as np
from sklearn.preprocessing import normalize

A = np.matrix([[0.80,0.15,0.09,0.19,0.11],[0.04,0.71,0.05,0.02,0.01],[0.03,0.07,0.82,0.01,0],[0.10,0.07,0,0.76,0.04],[0.03,0.01,0.03,0.03,0.85]])
A = normalize(A, norm='l1', axis=0)
limA = np.linalg.matrix_power(A,10**4)[:,0]
ctA = np.round(np.dot(A,np.diag(limA*706)))

B = np.matrix([[0.77,0.18,0.05,0.24,0.20],[0.10,0.75,0.05,0.02,0],[0.01,0.05,0.9,0,0],[0.10,0,0,0.72,0],[0.03,0.02,0,0.02,0.80]])
B = normalize(B, norm='l1', axis=0)
limB = np.linalg.matrix_power(B,10**4)[:,0]
ctB = np.round(np.dot(B,np.diag(limB*308)))

res = compareTransitions(ctA,ctB)
print(res)

[ 0.23497852  0.48053546]
